# Building a Spam Filter Using Naive Bayes

Our aim in this project is to build a spam filter for SMS messages using the multinomial naive Bayes classifier. At the end of this project we will have a program that classifies messages as spam or non-spam (ham) with an accuracy greater than 98%.

We are going to use a data set that contains 5,572 SMS messages that are already classified. The data set we will use, and more info on it, can be found on [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

## Exploring the Data Set

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Name the columns in a way that will not get mixed up with any actual words from messages later on
sms = pd.read_csv('files/SMSSpamCollection', sep='\t', header=None, names=['sms_label', 'sms_message'])
sms.head()

,sms_label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
sms.shape

(5572, 2)

In [4]:
sms['sms_label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: sms_label, dtype: float64

We see that about 87% of the messages in the data set are ham and the remaining are spam.

## Forming Training and Test Sets

We will split the data set into a training set and a test set, where the training set will have 80% of the data and the rest will be kept as a test set.

In [5]:
# The index for splitting
cutoff = round(len(sms) * 0.8)

# Randomize the data set before splitting
sms_randomized = sms.sample(frac=1, random_state=1)

training = sms_randomized[:cutoff].reset_index(drop=True)
test = sms_randomized[cutoff:].reset_index(drop=True)

print(training.shape)
print(test.shape)

(4458, 2)
(1114, 2)


Below we take a look at the percentages of spam and ham messages in both sets to check if they are close to the percentage in the full data set; we find that the results are very similar.

In [6]:
training['sms_label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: sms_label, dtype: float64

In [7]:
test['sms_label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: sms_label, dtype: float64

## Cleaning the Data

In order to calculate the probabilities used by the multinomial naive Bayes classifier, we first need to encode the messages in a format that makes it easy to extract information. We will form a vocabulary set for the messages in the training set, and we will create a column for each word in the vocabulary that keeps track of how many times the word appears in a message (note that each row corresponds to a message in our data set).

First we remove punctuation and change the letters to lower case in all messages.

In [8]:
training['sms_message'] = training['sms_message'].str.replace('\W', ' ', regex=True).str.lower()
training.head()

,sms_label,sms_message
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


Next we create a vocabulary, that is, a set containing all the unique words appearing in all messages.

In [9]:
training['sms_message'] = training['sms_message'].str.split()

vocabulary = []
for message in training['sms_message']:
    for word in message:
        vocabulary.append(word)

# Form a set to remove reccurences
vocabulary = list(set(vocabulary))

len(vocabulary)

7783

Now we use the vocabulary to add columns to our training set that will trace occurences of each word in each message.

In [10]:
# Dictionary mapping each word to the number of times it appears in each message
word_counts = {word: [0] * len(training) for word in vocabulary}

# Go through each message to count
for index, message in enumerate(training['sms_message']):
    for word in message:
        word_counts[word][index] += 1

word_counts = pd.DataFrame(word_counts)
word_counts.head()

,fatty,oli,blur,gona,october,hunting,showrooms,cstore,weirdest,enjoying,...,enjoyed,lifetime,tirupur,freezing,tap,oops,server,albi,burgundy,vegas
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
training = pd.concat([training, word_counts], axis=1)
training.head()

,sms_label,sms_message,fatty,oli,blur,gona,october,hunting,showrooms,cstore,...,enjoyed,lifetime,tirupur,freezing,tap,oops,server,albi,burgundy,vegas
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Using Multinomial Naive Bayes

Now that our training set is ready, we will start our calculations needed for the spam filter.

Recall that, given a message, the multinomial naive Bayes algorithm uses the following proportions to classify the message:

\begin{equation*}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation*}

\begin{equation*}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation*}

Here, to calculate $P(w_i|Spam)$ and $P(w_i|Ham)$, we will use Laplace smoothing ($\alpha = 1$).

\begin{equation*}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation*}

\begin{equation*}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation*}

We start by calculating the terms that recur for each word, that is, the ones independent of $w_i$. These are $P(Spam)$, $P(Ham)$, $N_{Spam}$, $N_{Ham}$ and $N_{Vocabulary}$.

In [12]:
spam = training[training['sms_label'] == 'spam']
ham = training[training['sms_label'] == 'ham']

p_spam = len(spam) / len(training)
p_ham = len(ham) / len(training)

n_spam = spam['sms_message'].apply(len).sum()
n_ham = ham['sms_message'].apply(len).sum()

n_vocabulary = len(vocabulary)

Next we calculate the conditional probabilities for each word, namely $P(w_i|Spam)$ and $P(w_i|Ham)$. We will use separate dictionaries for the spam and ham cases.

In [13]:
cond_spam, cond_ham = {}, {}

alpha = 1

for w in vocabulary:
    cond_spam[w] = (spam[w].sum() + alpha) / (n_spam + alpha * n_vocabulary)
    cond_ham[w] = (ham[w].sum() + alpha) / (n_ham + alpha * n_vocabulary)

Now we are ready to go ahead with the classification. Recall that the classifier compares the products $P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)$ and $P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)$.

In [14]:
import re

def classify(sms, print_posterior=False):
    '''sms: (str) an sms message
    returns: (str) classification of s
    '''
    s = re.sub('\W', ' ', sms)
    s = s.lower().split()
    
    p_spam_given_sms = p_spam
    p_ham_given_sms = p_ham
    
    for w in s:
        if w in cond_spam:
            p_spam_given_sms *= cond_spam[w]
        if w in cond_ham:
            p_ham_given_sms *= cond_ham[w]
    
    if print_posterior:
        print("SMS: \"{}\"".format(sms))
        print("P(Spam|SMS) = {}".format(p_spam_given_sms))
        print("P(Ham|SMS) = {}".format(p_ham_given_sms))
    
    if p_spam_given_sms > p_ham_given_sms:
        return 'spam'
    elif p_spam_given_sms < p_ham_given_sms:
        return 'ham'
    else:
        return 'unclassified'

Let's try a couple of messages:

In [15]:
test.head()

,sms_label,sms_message
0,ham,Later i guess. I needa do mcat study too.
1,ham,But i haf enuff space got like 4 mb...
2,spam,Had your mobile 10 mths? Update to latest Oran...
3,ham,All sounds good. Fingers . Makes it difficult ...
4,ham,"All done, all handed in. Don't know if mega sh..."


In [16]:
classify(test['sms_message'][0], print_posterior=True)

SMS: "Later i guess. I needa do mcat study too."
P(Spam|SMS) = 3.4831070937898343e-26
P(Ham|SMS) = 4.253245130534654e-19


'ham'

In [17]:
classify(test['sms_message'][2], print_posterior=True)

SMS: "Had your mobile 10 mths? Update to latest Orange camera/video phones for FREE. Save £s with Free texts/weekend calls. Text YES for a callback orno to opt out"
P(Spam|SMS) = 7.548549643070596e-83
P(Ham|SMS) = 4.338466063216561e-98


'spam'

## Conclusion: Measuring Accuracy

Now that we have built our spam filter, we will see how it performs on the test set. We will create a new column containing the output of our classifier, and then compare it with the original labels.

In [18]:
test['prediction'] = test['sms_message'].apply(classify)
test.head()

,sms_label,sms_message,prediction
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [19]:
correct = test['sms_label'] == test['prediction']
correct.value_counts()

True     1100
False      14
dtype: int64

In [20]:
correct.value_counts(normalize=True)

True     0.987433
False    0.012567
dtype: float64

The accuracy of our spam filter is close to 98.74%. Out of the 1114 messages in the test set, our filter misclassified only 14.